# Import

In [3]:
import csv
import os
import sys

import numpy as np
import pandas as pd
import umap
from scipy.sparse.csgraph import connected_components
from sklearn.decomposition import PCA
from tqdm import tqdm

## Add configuration file

In [4]:
sys.path.append("/home/jovyan/core/config/")
sys.path.append("/home/jovyan/core/util/")

In [5]:
from ALL import config 
from util import *

## Set condition

In [6]:
tqdm.pandas()
pd.set_option("display.max_columns", 100)
pd.set_option("display.max_rows", 50)

In [ ]:
s3 = S3Manager()

In [7]:
data_type="20News"

In [ ]:
transformer_model = "sentence-transformers/all-MiniLM-L6-v2"

# Read data

In [11]:
df = pd.read_csv(
    f"../../Preprocessing/data/{data_type}/master.csv", index_col=0
)

In [12]:
with open(f"../../Preprocessing/data/{data_type}/class.csv", mode="r") as f:
    reader = csv.reader(f)
    class_labels = [label for label in reader]

In [13]:
max_model_num = config["vectorize"]["sentenceBERT"]["max_model_num"]
vector_dims = config["vectorize"]["sentenceBERT"]["dims"]

# Dimension Depression

In [14]:
vector_path = f"../data/{data_type}/sentenceBERT/vector"

for vector_dim in tqdm(vector_dims):
    for model_num in range(max_model_num):
        vectors = np.load(
            f"{vector_path}/raw/{model_num}.npy",
        )
        reduced_vectors = PCA(
            n_components=vector_dim, random_state=model_num
        ).fit_transform(vectors)

        np.save(
            make_filepath(f"{vector_path}/pca/{vector_dim}/{model_num}.npy"),
            reduced_vectors,
        )

100%|██████████| 9/9 [00:05<00:00,  1.73it/s]


In [15]:
vector_path = f"../data/{data_type}/sentenceBERT/vector"

for vector_dim in tqdm(vector_dims):
    for model_num in range(max_model_num):
        vectors = np.load(
            f"{vector_path}/raw/{model_num}.npy",
        )
        reduced_vectors = umap.UMAP(
            n_components=vector_dim, random_state=model_num
        ).fit_transform(vectors)
        np.save(
            make_filepath(f"{vector_path}/umap/{vector_dim}/{model_num}.npy"),
            reduced_vectors,
        )

100%|██████████| 9/9 [07:22<00:00, 49.21s/it]
